In [1]:
# Usefull imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from hmmlearn import hmm
from IOHMM import IOHMM_model

In [2]:
hmm_model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=100)

hmm_model.startprob_ = np.array([0.2, 0.8])

hmm_model.transmat_ = np.array([[0.4, 0.6],
                                [0.2, 0.8]])

hmm_model.means_ = np.array([[-10.0], [10.0]])

hmm_model.covars_ = np.array([[.1], [.1]])


In [3]:
#sample data
X, Z = hmm_model.sample(5)

#dataframe
data = pd.DataFrame(X, columns=['X'])
data['Z'] = Z

data[:]

,X,Z
0,9.772072,1
1,9.628914,1
2,-9.851024,0
3,-9.568227,0
4,10.331455,1


In [4]:
# Torch  data
output = torch.tensor(data['X'].values, dtype=torch.float32)
# Input data ones
input = torch.ones(output.size(), dtype=torch.float32).unsqueeze(1)

In [5]:
# Transition matrix
TM = torch.tensor([[[1., 2.2],
                    [-1., 3.1]],
                    
                    [[1., 2.],
                    [-1.3, 3.]]], dtype=torch.float32)


IOHMM = IOHMM_model(num_states=2, inputs=input, outputs=output, max_iter=10, tol=1e-40)

In [6]:
IOHMM._baum_welch()

zeta[0]: tensor([0.5000, 0.5000])
iter: 0, likelihood: -24.135108947753906, likelihood marco: -97.72394561767578
zeta[0]: tensor([0.5001, 0.4999])
iter: 1, likelihood: -24.135173797607422, likelihood marco: -97.72383880615234
zeta[0]: tensor([0.5003, 0.4997])
iter: 2, likelihood: -24.135234832763672, likelihood marco: -97.72373962402344
zeta[0]: tensor([0.5004, 0.4996])
iter: 3, likelihood: -24.135295867919922, likelihood marco: -97.7236328125
zeta[0]: tensor([0.5005, 0.4995])
iter: 4, likelihood: -24.135353088378906, likelihood marco: -97.72352600097656
zeta[0]: tensor([0.5007, 0.4993])
iter: 5, likelihood: -24.135417938232422, likelihood marco: -97.72341918945312
zeta[0]: tensor([0.5008, 0.4992])
iter: 6, likelihood: -24.135475158691406, likelihood marco: -97.72331237792969
zeta[0]: tensor([0.5009, 0.4991])
iter: 7, likelihood: -24.135536193847656, likelihood marco: -97.72322082519531
zeta[0]: tensor([0.5010, 0.4990])
iter: 8, likelihood: -24.135597229003906, likelihood marco: -97.72

In [7]:
matrix=torch.tensor([[1,2,3],[4,5,6],[7,8,9]])
vector=torch.tensor([1,2,3])
print(vector.shape)
print(matrix.shape)
sum_vector=vector+matrix
print(sum_vector)
torch.sum(sum_vector, dim=0)

torch.Size([3])
torch.Size([3, 3])
tensor([[ 2,  4,  6],
        [ 5,  7,  9],
        [ 8, 10, 12]])


tensor([15, 21, 27])

In [8]:
IOHMM.viterbi()

tensor([1, 0, 0, 0, 1])